In [1]:
import autograd.numpy as np

## Dynamic Programming
Dynamic Programming is a central idea of control theory based on Principle of Optimality. Suppose the optimal solution for a problem passes through some intermediate points $(x_1,t_1)$, then the optimal solution to the same problem starting at $(x_1, t_1)$ must be the continuation of the same path.

This principle leads to numerical procedure called Dynamic Programming for solving multi-stage decision making problems

### Control Problem
\begin{align}
\min J = h\big(x(t_f)\big) &+ \int_{t_0}^{t_f} g\big( x(t), u(t), t\big) dt\\
\text{subject to:}\\
\dot{x} &= a(x,u, t)\\
x_{t_0} &= \text{fixed}\\
t_f &= \text{fixed}
\end{align}

### Solution
**Step 1**. Develop a grid over space/time. Look for possible final states $x_i(t_f)$ and evaluate final costs like discretizing the states into $n$ possible cases e.g. $[x_1, x_2, \cdots, x_n]$

**Step 2**. Evaluate the cost of control action by approximation of the integral of the cost.
Consider the scenario where there is state $x_i$ in time $t_k$ and apply control $u_k^{ij}$ to move to state $x_j$ at time $t_{k+1} = t_k + \triangledown t$.\
Approximate the the cost such as:

\begin{align}
\int_{t_k}^{t_{k+1}} g\big(x(t), u(t), t \big)dt \approx g\big( x_k^i, u_k^{ij}, t_k \big) \triangledown t
\end{align}

And solve for control inputs directly form the system model:
\begin{align}
x_{k+1}^j \approx x_k^i + \alpha \big(x_k^i, u_k^{ij}, t_k \big) \triangledown t \Rightarrow \alpha \big( x_k^i, u_k^{ij}, t_k \big) =\frac{x_{k+1}^j - x_k^i}{\triangledown t}
\end{align}

which can be solved to find $u_k^{ij}$

Process is simple if control inputs are affine:

\begin{align}
\dot{x}=f(x,t)+q(x,t)u
\end{align}

which gives:
\begin{align}
u_k^{ij} = q(x_k^i, t_k)^{-1}\left[ \frac{x_{k+1}^j - x_k^i}{\triangledown t} - f(x_k^i, t_k) \right]
\end{align}

So for any combination of $x_k^i$ and $x_{k+1}^j$ can evaluate the incremental cost $\triangledown J\big(x_k^i, x_{k+1}^j\big)$ of making the state transition.

Assuming the optimal path is known, each new terminal point $(x_{k+1}^j)$ can establish the optimal path to take from x_k^i using:

\begin{align}
J^*(x_k^i, t_k) = \min_{x_{k+1}^j} \big[ \triangledown J(x_k^i, x_{k+1}^j) + J^*(x_{k+1}^j) \big] 
\end{align}

Then for each $x_k^i$ the output is:
- Best $x_{k+1}^i$ to pick, because it gives lowest cost.
- Control input required to achieve the best cost.

Then work backwards the remaining states until $x_{t_0}$.

**Example 1** Shortest path problem (Bryson & Ho)
The goal is to travel from $A$ to $G$ in the shortest time possible.
- Travel times for each leg are shown in the figure
- There are 20 options to get from $A$ to $G$
- Or Start at N and work backwards, invoking the principle of optimality on each step.
- Using this alternative method we only solve 14 equations instead of 20 steps.
Consider the graph below for finding the shortest path.

![Shortest Path](images/shortest_path.jpg)

From visual inspection of the nodes and edges, it can be determined that the optimal paths are either $[A,C,F,G]$ and $[A,D,F,G]$ both having cost of 8 units.

Using the algebraic solution:

\begin{align}
J^*(v) &= \min_{w \in F_v} c(v,w) + J(w)\\
\textit{where:}&\\
J^*(v): &\text{ minimum cost to go from v to any node}\\
F_v: &\text{ set of adjacent nodes}\\
c(v,w): &\text{ cost of travelling from v to w}
\end{align}

**Solution A** Implement the Q matrix from the set generator below:
\begin{align}
Q(v, w) = \left\{\array{
c(v,w)& \text{ if } w \in F_v\\
+\infty& \text{ otherwise }
}\right.
\end{align}

\begin{align}
\left[\array{
v| &A      &B      &C      &D      &E      &F      &G\\
\hline
A| &\infty &1      &5      &3      &\infty &\infty &\infty\\
B| &1      &\infty &\infty &9      &6      &\infty &\infty\\
C| &5      &\infty &\infty &\infty &\infty &2      &\infty\\
D| &3      &9      &\infty &\infty &\infty &4      &8\\
E| &\infty &6      &\infty &\infty &\infty &\infty &4\\
F| &\infty &\infty &2      &4      &\infty &\infty &1\\
G| &\infty &6      &\infty &8      &4      &1      &\infty\\
}\right]
\end{align}

***Notes***:
- We are travelling from node A to G, hence the sequence ${J_n}$ with A as node 0.
- The cost to final destination $J(v=G) = 0$
- Cost to a non-destination node is $\infty$.

In [2]:
N = 7 # Node/Vertex size
inf = np.inf
Q = np.array([
    [inf, 1,   5,   3,   inf, inf, inf],
    [inf, inf, inf, 9,   6,   inf, inf],
    [inf, inf, inf, inf, inf, 2,   inf],
    [inf, inf, inf, inf, inf, 4,   8],
    [inf, inf, inf, inf, inf, inf, 4],
    [inf, inf, inf, inf, inf, inf, 1],
    [inf, inf, inf, inf, inf, inf, 0],
])

nodes = range(N)
J = np.zeros(N, dtype=int)
J_next = np.empty(N, dtype=int)
max_iter = 500
i = 0
while i < max_iter:
    for v in nodes:
        J_next[v] = np.min(Q[v,:] + J)
        #print(f"i:{i} node: {v} J={J_next}")
    if np.array_equal(J_next, J):
        #print("Destination reached!")
        break
    J[:] = J_next
    i += 1    
print (f"Cost-to-go function: J*={J}, {np.sum(J)}")

Cost-to-go function: J*=[ 8 10  3  5  4  1  0], 31


**Solution B**. Start at destionation node $G$ and **work backwards**, invoking the principle of optimality on each step.

\begin{align}
\left[\array{
v&| &G      &F      &E      &D      &C      &B      &A\\
\hline
G&| &\infty &1      &4      &8      &\infty &\infty &\infty\\
F&| &\infty &\infty &\infty &4      &2      &\infty &\infty\\
E&| &\infty &\infty &\infty &\infty &\infty &6      &\infty\\
D&| &\infty &\infty &\infty &\infty &\infty &9      &3\\
C&| &\infty &\infty &\infty &\infty &\infty &\infty &5\\
B&| &\infty &\infty &\infty &\infty &\infty &\infty &1\\
A&| &\infty &\infty &\infty &\infty &\infty &\infty &0\\
}\right]
\end{align}


In [3]:
N = 7 # Node/Vertex size
inf = np.inf
Q = np.array([
    [inf, 1,   4,   8,   inf, inf, inf],
    [inf, inf, inf, 4,   2,   inf, inf],
    [inf, inf, inf, inf, inf, 6,   inf],
    [inf, inf, inf, inf, inf, 9,   3],
    [inf, inf, inf, inf, inf, inf, 5],
    [inf, inf, inf, inf, inf, inf, 1],
    [inf, inf, inf, inf, inf, inf, 0],
])

nodes = range(N)
J = np.zeros(N, dtype=int)
J_next = np.empty(N, dtype=int)
max_iter = 500
i = 0
while i < max_iter:
    for v in nodes:
        J_next[v] = np.min(Q[v,:] + J)
        #print(f"i:{i} node: {v} J={path}")
    if np.array_equal(J_next, J):
        #print("Destination reached!")
        break
    J[:] = J_next
    i += 1    
print (f"Cost-to-go function: J*={J}, {np.sum(J)}")

Cost-to-go function: J*=[8 7 7 3 5 1 0], 31


### Reference
[1] [Dynamic Programming, wiki](https://en.wikipedia.org/wiki/Dynamic_programming#Dijkstra's_algorithm_for_the_shortest_path_problem)

[2] [Optimal Control Theory. D. Kirk](https://www.amazon.com/Optimal-Control-Theory-Introduction-Engineering/dp/0486434842)

[3] [Dynamic Programming, Richard Bellman, Dover](https://books.google.com.ph/books/about/Dynamic_Programming.html?id=CG7CAgAAQBAJ&redir_esc=y)

[4] [Dynamic Programming and Optimal Control, Vol 1 & 2, D.P. Bertsekas](https://www.mit.edu/~dimitrib/dpbook.html)

[5] [Applied Optimal Control. Bryson & Ho](https://books.google.com.ph/books/about/Applied_Optimal_Control.html?id=P4TKxn7qW5kC&redir_esc=y)